<a href="https://colab.research.google.com/github/Ashish10Das/Deep_Learning/blob/main/LSTM_Next_Word_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**This project is related to text generator. Here we will be developing supervised model , where we will define input text and corresponding output text.
On the defined dataset (input and output) , we will train LSTM model.**

In [118]:
Data_System ="""Data Integration
A recurring theme in this book has been that for any given problem, there are several
solutions, all of which have different pros, cons, and trade-offs. For example, when
discussing storage engines in Chapter 3, we saw log-structured storage, B-trees, and
column-oriented storage. When discussing replication in Chapter 5, we saw singleleader,
multi-leader, and leaderless approaches.
If you have a problem such as “I want to store some data and look it up again later,”
there is no one right solution, but many different approaches that are each appropriate
in different circumstances. A software implementation typically has to pick one
particular approach. It’s hard enough to get one code path robust and performing
well—trying to do everything in one piece of software almost guarantees that the
implementation will be poor.
Thus, the most appropriate choice of software tool also depends on the circumstances.
Every piece of software, even a so-called “general-purpose” database, is designed
for a particular usage pattern.
Faced with this profusion of alternatives, the first challenge is then to figure out the
mapping between the software products and the circumstances in which they are a
good fit. Vendors are understandably reluctant to tell you about the kinds of workloads
for which their software is poorly suited, but hopefully the previous chapters
have equipped you with some questions to ask in order to read between the lines and
better understand the trade-offs.
However, even if you perfectly understand the mapping between tools and circumstances
for their use, there is another challenge: in complex applications, data is often
used in several different ways. There is unlikely to be one piece of software that is
suitable for all the different circumstances in which the data is used, so you inevitably
end up having to cobble together several different pieces of software in order to provide
your application’s functionality.
Combining Specialized Tools by Deriving Data
For example, it is common to need to integrate an OLTP database with a full-text
search index in order to handle queries for arbitrary keywords. Although some databases
(such as PostgreSQL) include a full-text indexing feature, which can be sufficient
for simple applications [1], more sophisticated search facilities require specialist
information retrieval tools. Conversely, search indexes are generally not very suitable
as a durable system of record, and so many applications need to combine two different
tools in order to satisfy all of the requirements.
We touched on the issue of integrating data systems in “Keeping Systems in Sync” on
page 452. As the number of different representations of the data increases, the inte‐
490 | Chapter 12: The Future of Data Systems
gration problem becomes harder. Besides the database and the search index, perhaps
you need to keep copies of the data in analytics systems (data warehouses, or batch
and stream processing systems); maintain caches or denormalized versions of objects
that were derived from the original data; pass the data through machine learning,
classification, ranking, or recommendation systems; or send notifications based on
changes to the data.
Surprisingly often I see software engineers make statements like, “In my experience,
99% of people only need X” or “…don’t need X” (for various values of X). I think that
such statements say more about the experience of the speaker than about the actual
usefulness of a technology. The range of different things you might want to do with
data is dizzyingly wide. What one person considers to be an obscure and pointless
feature may well be a central requirement for someone else. The need for data integration
often only becomes apparent if you zoom out and consider the dataflows
across an entire organization.
Reasoning about dataflows
When copies of the same data need to be maintained in several storage systems in
order to satisfy different access patterns, you need to be very clear about the inputs
and outputs: where is data written first, and which representations are derived from
which sources? How do you get data into all the right places, in the right formats?
For example, you might arrange for data to first be written to a system of record database,
capturing the changes made to that database (see “Change Data Capture” on
page 454) and then applying the changes to the search index in the same order. If
change data capture (CDC) is the only way of updating the index, you can be confident
that the index is entirely derived from the system of record, and therefore consistent
with it (barring bugs in the software). Writing to the database is the only way
of supplying new input into this system.
Allowing the application to directly write to both the search index and the database
introduces the problem shown in Figure 11-4, in which two clients concurrently send
conflicting writes, and the two storage systems process them in a different order. In
this case, neither the database nor the search index is “in charge” of determining the
order of writes, and so they may make contradictory decisions and become permanently
inconsistent with each other.
If it is possible for you to funnel all user input through a single system that decides on
an ordering for all writes, it becomes much easier to derive other representations of
the data by processing the writes in the same order. This is an application of the state
machine replication approach that we saw in “Total Order Broadcast” on page 348.
Whether you use change data capture or an event sourcing log is less important than
simply the principle of deciding on a total order.
Data Integration | 491
Updating a derived data system based on an event log can often be made deterministic
and idempotent (see “Idempotence” on page 478), making it quite easy to recover
from faults.
Derived data versus distributed transactions
The classic approach for keeping different data systems consistent with each other
involves distributed transactions, as discussed in “Atomic Commit and Two-Phase
Commit (2PC)” on page 354. How does the approach of using derived data systems
fare in comparison to distributed transactions?
At an abstract level, they achieve a similar goal by different means. Distributed transactions
decide on an ordering of writes by using locks for mutual exclusion (see
“Two-Phase Locking (2PL)” on page 257), while CDC and event sourcing use a log
for ordering. Distributed transactions use atomic commit to ensure that changes take
effect exactly once, while log-based systems are often based on deterministic retry
and idempotence.
The biggest difference is that transaction systems usually provide linearizability (see
“Linearizability” on page 324), which implies useful guarantees such as reading your
own writes (see “Reading Your Own Writes” on page 162). On the other hand,
derived data systems are often updated asynchronously, and so they do not by default
offer the same timing guarantees.
Within limited environments that are willing to pay the cost of distributed transactions,
they have been used successfully. However, I think that XA has poor fault tolerance
and performance characteristics (see “Distributed Transactions in Practice” on
page 360), which severely limit its usefulness. I believe that it might be possible to
create a better protocol for distributed transactions, but getting such a protocol
widely adopted and integrated with existing tools would be challenging, and unlikely
to happen soon.
In the absence of widespread support for a good distributed transaction protocol, I
believe that log-based derived data is the most promising approach for integrating
different data systems. However, guarantees such as reading your own writes are useful,
and I don’t think that it is productive to tell everyone “eventual consistency is
inevitable—suck it up and learn to deal with it” (at least not without good guidance
on how to deal with it).
In “Aiming for Correctness” on page 515 we will discuss some approaches for implementing
stronger guarantees on top of asynchronously derived systems, and work
toward a middle ground between distributed transactions and asynchronous logbased systems.
Asynchronous derived data"""

In [119]:
import tensorflow  as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM

In [120]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([Data_System])

In [121]:
tokenizer.word_index

{'the': 1,
 'to': 2,
 'of': 3,
 'and': 4,
 'data': 5,
 'in': 6,
 'for': 7,
 'is': 8,
 'a': 9,
 'on': 10,
 'that': 11,
 'systems': 12,
 'different': 13,
 'you': 14,
 'be': 15,
 'order': 16,
 'are': 17,
 'which': 18,
 'it': 19,
 'software': 20,
 'with': 21,
 'page': 22,
 'derived': 23,
 'an': 24,
 'distributed': 25,
 'transactions': 26,
 'database': 27,
 'need': 28,
 'as': 29,
 'search': 30,
 'index': 31,
 'see': 32,
 'writes': 33,
 'all': 34,
 'log': 35,
 'such': 36,
 'one': 37,
 'often': 38,
 'system': 39,
 'or': 40,
 'i': 41,
 'this': 42,
 'storage': 43,
 'we': 44,
 'if': 45,
 'circumstances': 46,
 'approach': 47,
 'guarantees': 48,
 'so': 49,
 'they': 50,
 'about': 51,
 'tools': 52,
 'by': 53,
 'based': 54,
 'problem': 55,
 'there': 56,
 'several': 57,
 'have': 58,
 'some': 59,
 'do': 60,
 'between': 61,
 'use': 62,
 'your': 63,
 'two': 64,
 'from': 65,
 'changes': 66,
 'only': 67,
 'same': 68,
 'other': 69,
 'integration': 70,
 'has': 71,
 'example': 72,
 'when': 73,
 'chapter': 74,

In [122]:
len(tokenizer.index_word)

531

In [123]:
input_sequences=[]

for sentence in Data_System.split('\n'):
  token_list= tokenizer.texts_to_sequences([sentence])[0] # here we are storing the entire text in terms of tokenized value
  print(token_list)

  for i in range(1, len(token_list)):
    n_gram_sequence = token_list[:i+1]
    input_sequences.append(n_gram_sequence)

[5, 70]
[9, 174, 175, 6, 42, 176, 71, 101, 11, 7, 177, 178, 55, 56, 17, 57]
[179, 34, 3, 18, 58, 13, 180, 181, 4, 102, 103, 7, 72, 73]
[104, 43, 182, 6, 74, 183, 44, 75, 35, 184, 43, 185, 186, 4]
[187, 188, 43, 73, 104, 105, 6, 74, 189, 44, 75, 190]
[191, 192, 4, 193, 76]
[45, 14, 58, 9, 55, 36, 29, 194, 106, 2, 195, 59, 5, 4, 196, 19, 77, 197, 198, 78]
[56, 8, 199, 37, 79, 200, 80, 107, 13, 76, 11, 17, 81, 108]
[6, 13, 46, 9, 20, 109, 201, 71, 2, 202, 37]
[110, 47, 203, 204, 205, 2, 111, 37, 206, 207, 208, 4, 209]
[210, 2, 60, 211, 6, 37, 82, 3, 20, 212, 48, 11, 1]
[109, 112, 15, 113]
[213, 1, 114, 108, 214, 3, 20, 215, 216, 217, 10, 1, 46]
[218, 82, 3, 20, 115, 9, 49, 219, 220, 221, 27, 8, 222]
[7, 9, 110, 223, 224]
[225, 21, 42, 226, 3, 227, 1, 83, 116, 8, 117, 2, 118, 119, 1]
[120, 61, 1, 20, 228, 4, 1, 46, 6, 18, 50, 17, 9]
[84, 229, 230, 17, 231, 232, 2, 121, 14, 51, 1, 233, 3, 234]
[7, 18, 122, 20, 8, 235, 236, 80, 237, 1, 238, 239]
[58, 240, 14, 21, 59, 241, 2, 242, 6, 16, 2, 2

In [124]:
max_len = max([len(x) for x in input_sequences])

In [125]:
padded_sequences = pad_sequences(input_sequences, maxlen = max_len , padding= 'pre')

In [126]:
padded_sequences

array([[  0,   0,   0, ...,   0,   5,  70],
       [  0,   0,   0, ...,   0,   9, 174],
       [  0,   0,   0, ...,   9, 174, 175],
       ...,
       [  0,   0,   0, ..., 173, 531,  12],
       [  0,   0,   0, ...,   0, 173,  23],
       [  0,   0,   0, ..., 173,  23,   5]], dtype=int32)

In [127]:
padded_sequences.shape

(1221, 20)

In [128]:
x= padded_sequences[:,:-1]

In [129]:
x

array([[  0,   0,   0, ...,   0,   0,   5],
       [  0,   0,   0, ...,   0,   0,   9],
       [  0,   0,   0, ...,   0,   9, 174],
       ...,
       [  0,   0,   0, ...,   4, 173, 531],
       [  0,   0,   0, ...,   0,   0, 173],
       [  0,   0,   0, ...,   0, 173,  23]], dtype=int32)

In [130]:
y = padded_sequences[:,-1]

In [131]:
y

array([ 70, 174, 175, ...,  12,  23,   5], dtype=int32)

In [132]:
x.shape

(1221, 19)

In [133]:
y.shape

(1221,)

So, we have 1221 texts(like "Data Integration" is one text) . we have (19+1 , index logic) 20 max number of word in a text within the entire input which we have Given.
And , we have 531 unique word in the enite input sentences , which can be found out by finding len of tokenized words index .


Summary  :  

*   **Unique words in entire input sentences** : 531  
  (*len of tokenized word index*)
*   **Max of words in a text** : 20   ( *max len of tokenized text sequence *)

*  **Total of input texts (lines )** : 1221   ( *from the shape of X or Y*)




In [134]:
from tensorflow.keras.utils import to_categorical
y = to_categorical(y,num_classes=532)

In [135]:
y.shape

(1221, 532)

In [150]:
model = Sequential()
model.add(Embedding(532, 100))
model.add(LSTM(150, return_sequences = True))
model.add(LSTM(150))
model.add(Dense(532, activation='softmax'))

In [151]:
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

In [152]:
model.build(input_shape=(None, 20))

In [153]:
model.summary()

Model: "sequential_20"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_20 (Embedding)             │ (None, 20, 100)             │          53,200 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_28 (LSTM)                       │ (None, 20, 150)             │         150,600 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_29 (LSTM)                       │ (None, 150)                 │         180,600 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_18 (Dense)                     │ (None, 532)                 │          80,332 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 464,732 (1.77 MB)

 Trainable params: 464,732 (1.77 MB)

 Non-trainable params: 0 (0.00 B)

In [155]:
model.fit(x,y,epochs=100)

Epoch 1/100
39/39 ━━━━━━━━━━━━━━━━━━━━ 7s 72ms/step - accuracy: 0.0252 - loss: 6.1267
Epoch 2/100
39/39 ━━━━━━━━━━━━━━━━━━━━ 3s 74ms/step - accuracy: 0.0554 - loss: 5.6104
Epoch 3/100
39/39 ━━━━━━━━━━━━━━━━━━━━ 3s 71ms/step - accuracy: 0.0474 - loss: 5.5718
Epoch 4/100
39/39 ━━━━━━━━━━━━━━━━━━━━ 5s 123ms/step - accuracy: 0.0543 - loss: 5.4916
Epoch 5/100
39/39 ━━━━━━━━━━━━━━━━━━━━ 3s 72ms/step - accuracy: 0.0540 - loss: 5.5547
Epoch 6/100
39/39 ━━━━━━━━━━━━━━━━━━━━ 5s 71ms/step - accuracy: 0.0463 - loss: 5.5269
Epoch 7/100
39/39 ━━━━━━━━━━━━━━━━━━━━ 6s 95ms/step - accuracy: 0.0560 - loss: 5.5061
Epoch 8/100
39/39 ━━━━━━━━━━━━━━━━━━━━ 4s 72ms/step - accuracy: 0.0504 - loss: 5.4291
Epoch 9/100
39/39 ━━━━━━━━━━━━━━━━━━━━ 4s 101ms/step - accuracy: 0.0511 - loss: 5.3305
Epoch 10/100
39/39 ━━━━━━━━━━━━━━━━━━━━ 5s 136ms/step - accuracy: 0.0593 - loss: 5.2158
Epoch 11/100
39/39 ━━━━━━━━━━━━━━━━━━━━ 8s 73ms/step - accuracy: 0.0530 - loss: 5.1436
Epoch 12/100
39/39 ━━━━━━━━━━━━━━━━━━━━ 7s 108ms/

In [158]:
import time
import numpy as np
text = "order to handle"

for i in range(10):
  # tokenize
  token_text = tokenizer.texts_to_sequences([text])[0]
  # padding
  padded_token_text = pad_sequences([token_text], maxlen=20, padding='pre')
  # predict
  pos = np.argmax(model.predict(padded_token_text))

  for word,index in tokenizer.word_index.items():
    if index == pos:
      text = text + " " + word
      print(text)
      time.sleep(2)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
order to handle queries
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
order to handle queries for
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
order to handle queries for arbitrary
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
order to handle queries for arbitrary keywords
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
order to handle queries for arbitrary keywords although
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
order to handle queries for arbitrary keywords although some
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
order to handle queries for arbitrary keywords although some databases
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
order to handle queries for arbitrary keywords although some databases for
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
order to handle queries for arbitrary keywords although some databases for several
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
order to handle queries for arbitrary keywords although some databases for several storage
